# EXPLORACIÓN Y LIMPIEZA DEL DATASET

- Cargamos **librerías, funciones y el data set** que vamos a usar para la limpieza 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.simplefilter("ignore")

In [2]:
#ORDENAR - esta es la importacion del archivo de las funciones de limpieza que me he creado
import sys
sys.path.append("src/")
from src.cleaning_functions import *

In [3]:
ds = pd.read_csv("data/attacks.csv",encoding = "ISO-8859-1")

- Hacemos una copia del data set "just in case"...

In [4]:
df = ds.copy()

# EXPLORACIÓN DATA SET

![alt text](https://memegenerator.net/img/instances/40379228/let-me-take-a-look-at-this.jpg "Take a look")

In [5]:
df.shape #25.723 Lineas y 24 columnas

(25723, 24)

### Vamos a cambiar las opciones de pandas para poder ver todas las columnas del dataset mejor

In [13]:
pd.set_option.display[max_columns, None]

AttributeError: 'CallableDynamicDoc' object has no attribute 'display'

In [ ]:
df.sample(10)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
19661,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24519,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2689,1989.04.00,Apr-1989,1989.0,Unprovoked,USA,Hawaii,"Kekaha Beach, Kaua'i",Paddling on surfboard,William P. Allen,M,NaN,"Board rammed by shark, skegs knocked loose & 5...",N,NaN,NaN,"J. Borg, p.77; L. Taylor (1993), pp.108-109",1989.04.00-Allen.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1989.04.00,1989.04.00,3614.0,NaN,NaN
20083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20881,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null 

In [ ]:
df.describe().T #Solo hay dos columnas creadas como numéricas

,count,mean,std,min,25%,50%,75%,max
Year,6300.0,1927.272381,281.116308,0.0,1942.0,1977.0,2005.0,2018.0
original order,6309.0,3155.999683,1821.396206,2.0,1579.0,3156.0,4733.0,6310.0


In [ ]:
Nan_percentage = round(df.isna().sum()*100/len(df),2) #Calculo el porcentaje de valores NaN
Nan_percentage.sort_values(ascending = False)

Unnamed: 22               100.00
Unnamed: 23                99.99
Time                       88.54
Species                    86.53
Age                        86.51
Sex                        77.70
Activity                   77.62
Fatal (Y/N)                77.60
Location                   77.60
Area                       77.27
Name                       76.32
Country                    75.69
Injury                     75.61
Investigator or Source     75.57
Type                       75.52
Year                       75.51
Date                       75.50
pdf                        75.50
href formula               75.50
href                       75.50
Case Number.1              75.50
Case Number.2              75.50
original order             75.47
Case Number                66.17
dtype: float64

![alt text](https://miro.medium.com/max/413/0*Cir0TzUEkHMbb8QB "Cleaning data")

- Muchas columnas tienen demasiado porcentaje de NaN's por lo que podremos prescindir de ellas ya que no aportan información.
- Vamos a eliminar todas las filas que tienen NaN en todos sus campos

In [ ]:
df.dropna(axis=0, inplace= True, how='all') #Elimino las filas que tienen todos los valores NaN

In [ ]:
df.shape #Ahora tenemos 8.703 filas

(8703, 24)

- Hay muchas filas que tienen un porcentaje demasiado alto de NaN, por lo que vamos a eliminar también las filas que tienen un 80% de NaN en su contenido

In [ ]:
df.dropna(axis=0, inplace= True, thresh=int(24*0.8))

- Actualizamos el porcentaje de NaN's de cada columna:

In [ ]:
df.isnull().sum().apply(lambda x: x*100/df.shape[0]).sort_values(ascending=False) #Otro método parecido al anterior

Unnamed: 22               99.981252
Unnamed: 23               99.962505
Time                      45.406824
Species                   40.101237
Age                       36.164229
Fatal (Y/N)                6.786652
Sex                        4.068241
Activity                   2.868391
Location                   2.699663
Area                       1.931009
Name                       0.449944
Investigator or Source     0.093738
Country                    0.037495
Injury                     0.037495
Type                       0.018748
href formula               0.018748
Date                       0.000000
pdf                        0.000000
href                       0.000000
Case Number.1              0.000000
Case Number.2              0.000000
original order             0.000000
Year                       0.000000
Case Number                0.000000
dtype: float64

- Vamos a ver si hay alguna fila duplicada, en tal caso las eliminaremos

In [ ]:
df.duplicated().sum() #2392 duplicados

0

In [ ]:
df.iloc[8698:8702] #Son filas con todo NaN salvo el case number, las eliminaremos también

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23


In [ ]:
df.drop_duplicates(inplace=True) #Elimino las filas duplicadas
df.reset_index(drop=True, inplace=True) #Reiniciamos el índice

- Eliminamos las columnas 'Unnamed: 22' y 'Unnamed: 23', ya vimos antes que eran prácticamente 100% NaN

In [ ]:
df.drop(['Unnamed: 22','Unnamed: 23'], axis=1, inplace=True)

- Veámos como están escritos los nombres de las columnas

In [ ]:
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order'],
      dtype='object')

- Quitemos los espacios en blanco en los nombres de las columnas, capitalicemos los nombres y sustituyamos espacios por guiones bajos

In [ ]:
df.columns = df.columns.str.strip().str.capitalize().str.replace(' ', '_')

# LIMPIEZA

![alt text](https://miro.medium.com/max/568/1*S1HH5F8PqWWcId9sb0L8og.jpeg "Dropna")

- Quitamos todas las filas y columnas que tengan todo NaN

In [ ]:
df.columns

Index(['Case_number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal_(y/n)', 'Time',
       'Species', 'Investigator_or_source', 'Pdf', 'Href_formula', 'Href',
       'Case_number.1', 'Case_number.2', 'Original_order'],
      dtype='object')

- Echando una ojeada a estas columnas podemos descartarlas ya que no nos van a aportar al estudio que vamos a hacer

In [ ]:
print(df["Href"][3513],df['Href_formula'][3513])

http://sharkattackfile.net/spreadsheets/pdf_directory/1962.01.01-Caberto.pdf http://sharkattackfile.net/spreadsheets/pdf_directory/1962.01.01-Caberto.pdf


- Contienen vínculos a archivos pdf donde vienen registrados datos del ataque en pdf: imagenes, recortes de periódico, fotos de la ubicación, etc...

https://sharkattackfile.net/spreadsheets/pdf_directory/1967.08.25-Casucci.pdf

- Ojeadas las columnas podemos hacer **drop** de las que **no considero interesantes** para el caso:

``'Case_number','Investigator_or_source', 'Pdf', 'Href_formula', 'Href', 'Case_number.1', 'Case_number.2', 'Original_order'``

In [ ]:
df.drop(['Case_number','Investigator_or_source', 'Pdf', 'Href_formula', 'Href', 'Case_number.1', 'Case_number.2', 'Original_order'], axis=1, inplace=True)

In [ ]:
df.columns

Index(['Date', 'Year', 'Type', 'Country', 'Area', 'Location', 'Activity',
       'Name', 'Sex', 'Age', 'Injury', 'Fatal_(y/n)', 'Time', 'Species'],
      dtype='object')

### YEAR Y DATE COLUMN

In [ ]:
df.Year.unique() #Vamos a ver si podemos rescatar algun año a través del contenido de la columna date

array([2018., 2017., 2016., 2015., 2014., 2013., 2012., 2011., 2010.,
       2009., 2008., 2007., 2006., 2005., 2004., 2003., 2002., 2001.,
       2000., 1999., 1998., 1997., 1996., 1995., 1984., 1994., 1993.,
       1992., 1991., 1990., 1989., 1988., 1987., 1986., 1985., 1983.,
       1982., 1981., 1980., 1979., 1978., 1977., 1976., 1975., 1974.,
       1973., 1972., 1971., 1970., 1969., 1968., 1967., 1966., 1965.,
       1964., 1963., 1962., 1961., 1960., 1959., 1958., 1957., 1956.,
       1955., 1954., 1953., 1952., 1951., 1950., 1949., 1948., 1848.,
       1947., 1946., 1945., 1944., 1943., 1942., 1941., 1940., 1939.,
       1938., 1937., 1936., 1935., 1934., 1933., 1932., 1931., 1930.,
       1929., 1928., 1927., 1926., 1925., 1924., 1923., 1922., 1921.,
       1920., 1919., 1918., 1917., 1916., 1915., 1914., 1913., 1912.,
       1911., 1910., 1909., 1908., 1907., 1906., 1905., 1904., 1903.,
       1902., 1901., 1900., 1899., 1898., 1897., 1896., 1895., 1894.,
       1893., 1892.,

- Vemos años un poco raros, así que paso a verlos en detalle

In [ ]:
pd.set_option('max_rows', None)

In [ ]:
QUIETO

In [ ]:
df[['Date','Year']][df.Year == 0] #81 fechas que podemos rescatar a través de date

,Date,Year
5253,Ca. 336.B.C..,0.0
5254,Ca. 725 B.C.,0.0
5255,1990 or 1991,0.0
5256,Before 2016,0.0
5257,Before Oct-2009,0.0
5258,Before 1934,0.0
5259,2009?,0.0
5260,Before 1930,0.0
5261,1880-1899,0.0
5262,Before 1909,0.0


Hay 81 lineas con el df.Year == CERO pero en df.Date si que figuran datos, a ver que podemos rescatar de ahí.
Muchos corresponend con intervalos de años, otros indica before 'alguna fecha' y en otro habla sobre la guerra, suponemos que la Segunda Guerra Mundial

In [ ]:
df.Date = df.Date.replace(regex=r'(?i)Reported\s{1,9}',value='')
list(df.Date.unique())

['25-Jun-2018',
 '18-Jun-2018',
 '09-Jun-2018',
 '08-Jun-2018',
 '04-Jun-2018',
 '03-Jun-2018',
 '27-May-2018',
 '26-May-2018',
 '24-May-2018',
 '21-May-2018',
 '13-May-2018',
 'May 2018',
 '12-May-2018',
 '09-May-2018',
 '30-Apr-2018',
 '28-Apr-2018',
 '25-Apr-2018',
 '24-Apr-2018',
 '23-Apr-2018',
 '22-Apr-2018',
 '19-Apr-2018',
 '15-Apr-2018',
 '14-Apr-2018',
 '10-Apr-2018',
 '05-Apr-2018',
 '03-Apr-2018',
 '31-Mar-2018',
 '14-Mar-2018',
 '9-Mar-2018',
 '24-Feb-2018',
 '23-Feb-2018',
 '18-Feb-2018',
 '15-Feb-2018',
 '14-Feb-2018',
 '11-Feb-2018',
 '03-Feb-2018',
 '01-Feb-2018',
 '28-Jan-2018',
 '21-Jan-2018',
 '14-Jan-2018',
 '13-Jan-2018',
 '12-Jan-2018',
 '05-Jan-2018',
 '31-Dec-2017',
 '30-Dec-2017',
 '21-Dec-2017',
 '09-Dec-2017',
 '30-Nov-2017',
 '24-Nov-2017',
 '18-Nov-2017',
 '13-Nov-2017',
 '04-Nov-2017',
 '31-Oct-2017',
 '28-Oct-2017',
 '26-Oct-2017',
 '23-Oct-2017',
 '22-Oct-2017',
 '21-Oct-2017',
 '18-Oct-2017',
 '09-Oct-2017',
 '05-Oct-2017',
 '01-Oct-2017',
 '25-Sep-201

- Aunque no salen en los **uniques** hay muchos Dates que salen como rangos, o datos de antes de Cristo.
- Vamos a denominarlos momentáneamente salvables para ver cuantos hay exactamente:

In [ ]:
salvables = df.loc[(df["Year"] == 0) & (df["Date"] != np.nan)]
salvables.shape

(81, 14)

In [ ]:
df.describe()

,Year
count,5334.000000
mean,1943.141545
std,244.938571
min,0.000000
25%,1951.250000
50%,1985.000000
75%,2007.000000
max,2018.000000


- Llamamos a las funciones definidas en el **cleanin_functions.py**, mas concretamente a rescatar fechas que aplica 3 funciones secuencialmente donde coge por orden los que contienen BC, los que son fechas sueltas (tipo Before YYYY) y luego los intervalos de los cuales saca la media, para rellenar los datos de df.Year para esos valores de df.Dates  #TODO, mira tema de late´s y demas para meterlos tb

In [ ]:
df.Year = df.Date.apply(rescatar_fechas)

- Si miramos el dato primero de nuestra lista anterior vemos que:

In [ ]:
df["Date"][6228]

KeyError: 6228

In [ ]:
df["Year"][6228]

In [ ]:
df["Date"][6265]

In [ ]:
df["Year"][6265]

In [ ]:
#pd.set_option('max_rows', None)

In [ ]:
df[["Date","Year"]].sample(10)

In [ ]:
#pd.set_option('max_rows', 20)

## AREA / LOCATION COLUMNS

- Ambas columnas son bastante imprecisas en cuanto a ubicacion, por lo que mejor voy a trabajar sobre la columna **country.**

In [ ]:
df[["Country","Area","Location"]].sample(10)

- Country voy a limpiar con una funcion metidad en cleaning_functios.py llamada paises. He cogido una lista de paises de Github que esta
metida en un CSV, lo que hace la funcion es mirar si la cadena correspondiente está en el CSV y sino le asigna NaN adema de un par de incorrecciones de la escritura de nombres que tambien he metido

In [ ]:
df.Country = df.Country.apply(paises)

In [ ]:
df.Country.notna().sum()

In [ ]:
#Esto se puede mejorar, para resultados sea...

# ACTIVITY COLUMN

In [ ]:
df.Activity.sample(30)

- La lista de actividades es amplia, vamos a agruparlas y filtrarlas a través de una función llamada actividad tambien contenida en cleaning_functions.py

In [ ]:
df.Activity = df.Activity.apply(actividad)
df.Activity.sample(30)

# COLUMNA INJURY

In [ ]:
list(df["Fatal (Y/N)"].unique())

In [ ]:
df[["Fatal (Y/N)","Injury"]].sample(30)

- El tema de valores distintos de los previsibles "Y"/ "N" ademas de categorizar la columna injury lo vamos a hacer a través de dos funciones

In [ ]:
df["Injury"] = df["Injury"].apply(lesiones)

In [ ]:
df['Fatal (Y/N)'] = df["Fatal (Y/N)"].apply(fatal)

In [ ]:
df[["Fatal (Y/N)","Injury"]].sample(10)

In [ ]:
#TODO age

# SPECIES COLUMN

In [ ]:
df.Species.sample(30)

- Para limpiar esto vamos a usar algo parecido a lo que hemos hecho con activity para categorizar y leer las cadenas de dentro.Usaré la lista de nombres de https://sharkattackfile.net/species.htm
 que es de donde viene nuestro dataframe además

In [ ]:
df.Species = df.Species.apply(species)

In [ ]:
df.Species.sample(30)

# Exportamos el data frame limpio a un CSV nuevo

In [ ]:
df.to_csv("src/attack_limpio.csv",index=False)

- La visualizacion continua en `analysis.ipynb` [📑](analysis.ipynb) 